In [1]:
import requests
import json
import os
from dotenv import load_dotenv
import pandas as pd
from datetime import datetime
import folium
import time
import seaborn as sns
import plotly.express as px
import requests
import scrapy
from scrapy import Selector
from scrapy.crawler import CrawlerProcess
import logging





In [2]:
load_dotenv()

Open_Weather_API_Key_Myriam= os.getenv("Open_Weather_API_Key_Myriam") 

# Extraction des informations géographiques des 35 plus belles villes

Sur la liste des 35 plus belles villes, nous avons 34 noms de villes + 1 lieu d'interêt qui n'est pas une ville (Gorges du Verdon). Pour récupérer les latitudes et longitudes des 34 villes nous pouvons facilement utiliser l'API openweathermap. Cette API permet facilement de collecter également le pays et la région, ce qui est utile pour contrôler d'un même coût les éventuelles erreures liées à des villes homonymes dans le monde.
Cependant, cette API ne reconnait pas les Gorges du Verdon. Pour palier ce problème, une seconde API a été utilisée, nominatim.openstreetmap. Le nombre de requete est plus limité et elle ne permet pas de récupérer la région et le pays.

In [3]:
best_cities= ["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle",
#"Gorges du Verdon" exclu de la liste dans un premier temps car n'est pas une ville mais un point d'intérêt.
]


In [4]:
best_cities[-1]

'La Rochelle'

In [5]:
#création d'un dataframe avec la liste des best cities
df = pd.DataFrame(data = best_cities, columns = ["cities"])
df

,cities
0,Mont Saint Michel
1,St Malo
2,Bayeux
3,Le Havre
4,Rouen
5,Paris
6,Amiens
7,Lille
8,Strasbourg
9,Chateau du Haut Koenigsbourg


## Import des longitudes & latitudes des 35 best cities

In [6]:
df.shape

(34, 1)

In [7]:
for i,best_city in enumerate(best_cities):
    r = requests.get(f"http://api.openweathermap.org/geo/1.0/direct?q={best_city}&limit=1&appid={Open_Weather_API_Key_Myriam}")
    if r.status_code == 200:
        data = r.json()
  
        df.loc[i,"name"] = data[0]["name"] #pour s'assurer de la bonne attribution de la ville
        df.loc[i,"country"] = data[0]["country"] #pour vérifier la localisation
        df.loc[i,"state"] = data[0]["state"] # pour vérifier la localisation
        df.loc[i,"lat"] = data[0]["lat"]
        df.loc[i,"lon"] = data[0]["lon"]
    
    else:
        print(f"Erreur lors de la requête pour {best_cities} :", r.status_code)

df

,cities,name,country,state,lat,lon
0,Mont Saint Michel,Mont Saint-Michel,FR,Normandy,48.635954,-1.511460
1,St Malo,Saint-Malo,FR,Brittany,48.649518,-2.026041
2,Bayeux,Bayeux,FR,Normandy,49.276462,-0.702474
3,Le Havre,Le Havre,FR,Normandy,49.493898,0.107973
4,Rouen,Rouen,FR,Normandy,49.440459,1.093966
5,Paris,Paris,FR,Ile-de-France,48.858890,2.320041
6,Amiens,Amiens,FR,Hauts-de-France,49.894171,2.295695
7,Lille,Lille,FR,Hauts-de-France,50.636565,3.063528
8,Strasbourg,Strasbourg,FR,Grand Est,48.584614,7.750713
9,Chateau du Haut Koenigsbourg,Château du Haut-Kœnigsbourg,FR,Grand Est,48.249523,7.345492


In [8]:
url = "https://nominatim.openstreetmap.org/search"

headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'    }

lieux = ["Gorges du Verdon"]

for lieu in lieux:
    params = {
        'q': lieu,
        'format': 'json',
        'limit': 1
    }

response = requests.get(url, params=params, headers=headers)
response

<Response [200]>

In [9]:
# pour trouver les coordonnés des Gorges du Verdon, utiisation d'une seconde API (Elle est plus restrictive en termes du nombre de requete, c'est pour celà que la préédente API a été favorisée)
def rechercher_coordonnees(lieux):
    url = "https://nominatim.openstreetmap.org/search"
    coordonnees = []

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'    }

    for lieu in lieux:
        params = {
            'q': lieu,
            'format': 'json',
            'limit': 1
        }

        response = requests.get(url, params=params, headers=headers)

        if response.status_code == 200:
            data = response.json()
            if data:
                address = data[0].get('address', {})
                coordonnees.append({
                    'cities': lieu,
                    'name': address.get('city'),
                    'country': address.get('country'), #information non trouvée avec l'API
                    'state': address.get('state'), #information non trouvée avec l'API
                    'lat': data[0].get('lat'),
                    'lon': data[0].get('lon'),
                })
            else:
                coordonnees.append({
                    'cities': lieu,
                    'name': None,
                    'country': None,
                    'state': None,
                    'lat': None,
                    'lon': None

                })
        else:
            print(f"Erreur lors de la requête pour {lieu} :", response.status_code)
            coordonnees.append({
                    'cities': lieu,
                    'name': None,
                    'country': None,
                    'state': None,
                    'lat': None,
                    'lon': None,
            })

        # Attendre un peu entre les requêtes pour éviter de dépasser les limites
        time.sleep(1)

    return coordonnees


coordonnees = rechercher_coordonnees(["Gorges du Verdon"])
print(coordonnees)


[{'cities': 'Gorges du Verdon', 'name': None, 'country': None, 'state': None, 'lat': '43.7496562', 'lon': '6.3285616'}]


In [10]:
#réalisation d'un dataframe avec le même colonage que le dataframe initial
gdv = pd.DataFrame(coordonnees)
gdv

,cities,name,country,state,lat,lon
0,Gorges du Verdon,None,None,None,43.7496562,6.3285616


In [11]:
#Fusion des deux dataframes
df_merged = pd.concat([df, gdv], ignore_index=True)
df_merged

,cities,name,country,state,lat,lon
0,Mont Saint Michel,Mont Saint-Michel,FR,Normandy,48.635954,-1.51146
1,St Malo,Saint-Malo,FR,Brittany,48.649518,-2.026041
2,Bayeux,Bayeux,FR,Normandy,49.276462,-0.702474
3,Le Havre,Le Havre,FR,Normandy,49.493898,0.107973
4,Rouen,Rouen,FR,Normandy,49.440459,1.093966
5,Paris,Paris,FR,Ile-de-France,48.85889,2.320041
6,Amiens,Amiens,FR,Hauts-de-France,49.894171,2.295695
7,Lille,Lille,FR,Hauts-de-France,50.636565,3.063528
8,Strasbourg,Strasbourg,FR,Grand Est,48.584614,7.750713
9,Chateau du Haut Koenigsbourg,Château du Haut-Kœnigsbourg,FR,Grand Est,48.249523,7.345492


In [12]:
# Création d'une carte pour visualiser les 35 best cities/points d'intérêts et leur localisation exacte
map = folium.Map(location=[46.603354, 1.888334], zoom_start=6)

# Définir le marker 
icon_url = "https://images.icon-icons.com/3565/PNG/512/tree_flower_plant_nature_landscape_icon_225325.png"

# Ajouter les villes à la carte
for i, row in df_merged.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=row['cities'],
        icon=folium.CustomIcon(icon_url, icon_size=(40, 40))  # Ajustez la taille de l'icône si nécessaire
    ).add_to(map)

# Sauvegarder la carte dans un fichier HTML
map.save('map_best_city.html')

# Afficher la carte
map

In [13]:
# Enregistrement du DataFrame fusionné en fichier CSV
df_merged.to_csv('data/best_cities_merged.csv', index=False)

## Import weather information

In [14]:
df=pd.read_csv("data/best_cities_merged.csv")

In [15]:
url = f"https://api.openweathermap.org/data/3.0/onecall?lat={df.loc[0, 'lat']}&lon={df.loc[0, 'lon']}&units=metric&exclude=hourly,daily&appid={Open_Weather_API_Key_Myriam}"

r = requests.get(url)
data = r.json()
r

<Response [200]>

In [16]:
# import des informations météorologiques importantes
for i in df.index:
    lat = df.loc[i, 'lat']
    lon = df.loc[i, 'lon']

    url = f"https://api.openweathermap.org/data/3.0/onecall?lat={lat}&lon={lon}&units=metric&exclude=hourly,daily&appid={Open_Weather_API_Key_Myriam}"

    r = requests.get(url)

    if r.status_code == 200:
        data = r.json()
  
        df.loc[i,"date"] = data["current"]["dt"]
        df.loc[i, "temp"] = data["current"]["temp"]
        df.loc[i, "feels_like"] = data["current"]["feels_like"]
        df.loc[i, "uvi"] = data["current"]["uvi"]
        df.loc[i, "humidity"] = data["current"]["humidity"]
        df.loc[i, "wind_speed"] = data["current"]["wind_speed"]
        df.loc[i, "description"] = data["current"]["weather"][0]["description"]
        
    else:
        print(f"Erreur lors de la requête pour l'index {i} : {r.status_code}")
        print(r.text)  # Imprime le message d'erreur pour plus de détails



In [17]:
df.head()

,cities,name,country,state,lat,lon,date,temp,feels_like,uvi,humidity,wind_speed,description
0,Mont Saint Michel,Mont Saint-Michel,FR,Normandy,48.635954,-1.511460,1.745405e+09,12.99,12.64,3.26,88.0,9.01,broken clouds
1,St Malo,Saint-Malo,FR,Brittany,48.649518,-2.026041,1.745405e+09,12.29,12.03,4.91,94.0,7.20,broken clouds
2,Bayeux,Bayeux,FR,Normandy,49.276462,-0.702474,1.745405e+09,14.14,13.93,4.51,89.0,4.76,broken clouds
3,Le Havre,Le Havre,FR,Normandy,49.493898,0.107973,1.745405e+09,12.51,12.27,4.50,94.0,6.69,broken clouds
4,Rouen,Rouen,FR,Normandy,49.440459,1.093966,1.745405e+09,11.52,10.97,4.53,86.0,6.17,broken clouds


In [18]:
#conversion de la colonne date en formation datetime
df['date'] = pd.to_datetime(df['date'], unit='s')
df.head()

,cities,name,country,state,lat,lon,date,temp,feels_like,uvi,humidity,wind_speed,description
0,Mont Saint Michel,Mont Saint-Michel,FR,Normandy,48.635954,-1.511460,2025-04-23 10:39:55,12.99,12.64,3.26,88.0,9.01,broken clouds
1,St Malo,Saint-Malo,FR,Brittany,48.649518,-2.026041,2025-04-23 10:40:00,12.29,12.03,4.91,94.0,7.20,broken clouds
2,Bayeux,Bayeux,FR,Normandy,49.276462,-0.702474,2025-04-23 10:40:00,14.14,13.93,4.51,89.0,4.76,broken clouds
3,Le Havre,Le Havre,FR,Normandy,49.493898,0.107973,2025-04-23 10:40:01,12.51,12.27,4.50,94.0,6.69,broken clouds
4,Rouen,Rouen,FR,Normandy,49.440459,1.093966,2025-04-23 10:40:01,11.52,10.97,4.53,86.0,6.17,broken clouds


In [19]:
# Enregistrer le DataFrame en fichier CSV
df.to_csv('data/best_cities_weather.csv', index=False)

# Rapide EDA 

In [20]:
bc = pd.read_csv("data/best_cities_weather.csv")
bc

,cities,name,country,state,lat,lon,date,temp,feels_like,uvi,humidity,wind_speed,description
0,Mont Saint Michel,Mont Saint-Michel,FR,Normandy,48.635954,-1.511460,2025-04-23 10:39:55,12.99,12.64,3.26,88.0,9.01,broken clouds
1,St Malo,Saint-Malo,FR,Brittany,48.649518,-2.026041,2025-04-23 10:40:00,12.29,12.03,4.91,94.0,7.20,broken clouds
2,Bayeux,Bayeux,FR,Normandy,49.276462,-0.702474,2025-04-23 10:40:00,14.14,13.93,4.51,89.0,4.76,broken clouds
3,Le Havre,Le Havre,FR,Normandy,49.493898,0.107973,2025-04-23 10:40:01,12.51,12.27,4.50,94.0,6.69,broken clouds
4,Rouen,Rouen,FR,Normandy,49.440459,1.093966,2025-04-23 10:40:01,11.52,10.97,4.53,86.0,6.17,broken clouds
5,Paris,Paris,FR,Ile-de-France,48.858890,2.320041,2025-04-23 10:40:01,11.32,10.91,4.52,92.0,5.66,light intensity drizzle
6,Amiens,Amiens,FR,Hauts-de-France,49.894171,2.295695,2025-04-23 10:40:01,10.10,9.57,1.10,92.0,4.43,overcast clouds
7,Lille,Lille,FR,Hauts-de-France,50.636565,3.063528,2025-04-23 10:40:01,9.37,6.07,0.82,71.0,7.20,light intensity drizzle
8,Strasbourg,Strasbourg,FR,Grand Est,48.584614,7.750713,2025-04-23 10:40:01,17.01,16.41,3.67,63.0,2.06,clear sky
9,Chateau du Haut Koenigsbourg,Château du Haut-Kœnigsbourg,FR,Grand Est,48.249523,7.345492,2025-04-23 10:40:01,12.90,12.00,5.07,67.0,1.03,clear sky


In [21]:
bc.shape

(35, 13)

In [22]:
df_sorted = bc.sort_values(by='temp', ascending=True)

# Créer un diagramme en bâtons horizontal
fig = px.bar(df_sorted, x='temp', y='cities', orientation='h',
             title='Températures par Ville',
             labels={'temp': 'Température (°C)', 'cities': 'Ville'})

# Ajuster la hauteur du graphique
fig.update_layout(height=800)

# Afficher le diagramme
fig.show()

In [23]:
df_sorted = bc.sort_values(by='feels_like', ascending=True)

# Créer un diagramme en bâtons horizontal
fig = px.bar(df_sorted, x='feels_like', y='cities', orientation='h',
             title='Températures ressenties par Ville',
             labels={'feels_like': 'Température ressentie (°C)', 'cities': 'Ville'})

# Ajuster la hauteur du graphique
fig.update_layout(height=800)

# Afficher le diagramme
fig.show()

In [24]:
df_sorted = bc.sort_values(by='uvi', ascending=True)

# Créer un diagramme en bâtons horizontal
fig = px.bar(df_sorted, x='uvi', y='cities', orientation='h',
             title='indices UV par Ville',
             labels={'uvi': 'Indice UV', 'cities': 'Ville'})

# Ajuster la hauteur du graphique
fig.update_layout(height=800)

# Afficher le diagramme
fig.show()

In [25]:
df_sorted = bc.sort_values(by='humidity', ascending=False)

# Créer un diagramme en bâtons horizontal
fig = px.bar(df_sorted, x='humidity', y='cities', orientation='h',
             title='Humidité par Ville',
             labels={'humidity': 'Humidité', 'cities': 'Ville'})

# Ajuster la hauteur du graphique
fig.update_layout(height=800)

# Afficher le diagramme
fig.show()

In [26]:
df_sorted = bc.sort_values(by='wind_speed', ascending=False)

# Créer un diagramme en bâtons horizontal
fig = px.bar(df_sorted, x='wind_speed', y='cities', orientation='h',
             title='Vitesse du vent par Ville',
             labels={'wind_speed': 'wind_speed', 'cities': 'Ville'})

# Ajuster la hauteur du graphique
fig.update_layout(height=800)

# Afficher le diagramme
fig.show()

In [30]:
# Dictionnaire pour mapper les descriptions aux icônes
icon_mapping = {
    'clear sky': "https://images.icon-icons.com/2791/PNG/512/sunny_day_weather_sun_icon_177555.png",  
    'scattered clouds': 'https://images.icon-icons.com/2791/PNG/512/partly_cloudy_day_sun_clouds_weather_icon_177560.png',
    'few clouds' : 'https://images.icon-icons.com/857/PNG/512/weather-2_icon-icons.com_67742.png',
    'broken clouds': 'https://images.icon-icons.com/2791/PNG/512/cloudy_cloud_weather_icon_177580.png',
    'overcast clouds': 'https://images.icon-icons.com/2791/PNG/512/overcast_cloud_weather_icon_177561.png',
    'light intensity drizzle rain' : 'https://images.icon-icons.com/139/PNG/512/drizzle_day_weather_20907.png',
    'light intensity drizzle': 'https://images.icon-icons.com/139/PNG/512/drizzle_day_weather_20907.png',
    'mist': 'https://images.icon-icons.com/2791/PNG/512/fog_weather_icon_177578.png',
    'light rain' : 'https://images.icon-icons.com/2791/PNG/512/rain_cloud_weather_icon_177568.png',
    'heavy rain' : 'https://images.icon-icons.com/2791/PNG/512/heavy_rain_weather_icon_177574.png',
    'heavy intensity rain' : "https://images.icon-icons.com/2791/PNG/512/heavy_rain_day_weather_icon_177573.png",
    'moderate rain' : 'https://images.icon-icons.com/2791/PNG/512/heavy_rain_day_weather_icon_177573.png',
    'thunder storm' : 'https://images.icon-icons.com/2791/PNG/512/thunder_thunderbolt_cloud_weather_icon_177554.png',
    'snow' : 'https://images.icon-icons.com/2791/PNG/512/heavy_snow_weather_icon_177571.png'
}

# Créer une carte centrée sur la France
map = folium.Map(location=[46.603354, 1.888334], zoom_start=6)

# Ajouter les villes à la carte avec des icônes personnalisées
for i, row in bc.iterrows():
    icon_url = icon_mapping.get(row['description'], 'https://images.icon-icons.com/1744/PNG/512/3643775-caution-exclamation-mark-sign-triangle_113444.png')  #si non trouvé dans le dico, ce logo apparait
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=row['cities'],
        icon=folium.CustomIcon(icon_url, icon_size=(40, 40))  # Ajustez la taille de l'icône si nécessaire
    ).add_to(map)

# Sauvegarder la carte dans un fichier HTML
map.save('map_meteo_best_city.html')

# Afficher la carte
map


# Selection des meilleures destinations

Nous sommes au printemps. Nous cherchons une distination qui nous changerait de la grisaille parisienne. Pour définir les villes avec la meilleure météo, nous pouvons définir les conditions suivantes :
- ciel dégagé
- hautes température rééel et ressentie
- Indice UV élevé
- faible taux d'humidité
- faible vitesse du vent

In [31]:
bc[bc["description"]=="clear sky"]

,cities,name,country,state,lat,lon,date,temp,feels_like,uvi,humidity,wind_speed,description
8,Strasbourg,Strasbourg,FR,Grand Est,48.584614,7.750713,2025-04-23 10:40:01,17.01,16.41,3.67,63.0,2.06,clear sky
9,Chateau du Haut Koenigsbourg,Château du Haut-Kœnigsbourg,FR,Grand Est,48.249523,7.345492,2025-04-23 10:40:01,12.90,12.00,5.07,67.0,1.03,clear sky
10,Colmar,Colmar,FR,Grand Est,48.077752,7.357964,2025-04-23 10:40:01,15.94,15.34,5.07,67.0,1.03,clear sky
11,Eguisheim,Eguisheim,FR,Grand Est,48.044797,7.307962,2025-04-23 10:40:02,15.90,15.30,5.07,67.0,1.03,clear sky
17,Bormes les Mimosas,Bormes-les-Mimosas,FR,Provence-Alpes-Côte d'Azur,43.150697,6.341928,2025-04-23 10:40:02,18.55,18.05,6.34,61.0,6.17,clear sky
18,Cassis,Cassis,FR,Provence-Alpes-Côte d'Azur,43.214036,5.539632,2025-04-23 10:40:02,19.58,18.90,6.29,50.0,9.08,clear sky
19,Marseille,Marseille,FR,Provence-Alpes-Côte d'Azur,43.296174,5.369953,2025-04-23 10:40:02,19.75,19.09,6.19,50.0,7.20,clear sky
20,Aix en Provence,Aix-en-Provence,FR,Provence-Alpes-Côte d'Azur,43.529842,5.447474,2025-04-23 10:40:03,18.01,16.91,5.81,40.0,6.22,clear sky
21,Avignon,Avignon,FR,Provence-Alpes-Côte d'Azur,43.949249,4.805901,2025-04-23 10:40:03,19.67,19.02,6.14,51.0,6.17,clear sky
23,Nimes,Nimes,FR,Occitania,43.837425,4.360069,2025-04-23 10:40:03,20.22,19.26,6.22,37.0,4.63,clear sky


In [32]:
# Filtrer les villes avec la description "clear sky"
filtered_bc = bc[bc['description'] == 'clear sky']

# Sélectionner les 5 meilleures villes pour chaque critère parmi les villes filtrées.
top_feels_like = set(filtered_bc.nlargest(5, 'feels_like')['cities'])# critère le plus important, donc le plus restrictif
#top_temp = set(filtered_bc.nlargest(10, 'temp')['cities']) #critère retiré car redondant avec la précédent
top_uvi = set(filtered_bc.nlargest(10, 'uvi')['cities'])
top_humidity = set(filtered_bc.nsmallest(10, 'humidity')['cities'])
top_wind_speed = set(filtered_bc.nsmallest(15, 'wind_speed')['cities'])

# Trouver l'intersection des ensembles
best_weather = top_feels_like &  top_uvi & top_humidity & top_wind_speed
#top_temp &

# Afficher la liste des meilleures villes
print("Villes avec la meilleure météo :")
print(best_weather)

Villes avec la meilleure météo :
{'Marseille', 'Nimes', 'Avignon', 'Cassis', 'Gorges du Verdon'}


In [33]:
best_weather = list(best_weather) #transformer le set en liste
best_weather_df = bc[bc['cities'].isin(best_weather)] #puis filter le dataframe sur la liste
best_weather_df = best_weather_df.sort_values(by='feels_like', ascending=False)
best_weather_df


,cities,name,country,state,lat,lon,date,temp,feels_like,uvi,humidity,wind_speed,description
34,Gorges du Verdon,NaN,NaN,NaN,43.749656,6.328562,2025-04-23 10:40:04,19.98,19.42,6.38,53.0,3.53,clear sky
23,Nimes,Nimes,FR,Occitania,43.837425,4.360069,2025-04-23 10:40:03,20.22,19.26,6.22,37.0,4.63,clear sky
19,Marseille,Marseille,FR,Provence-Alpes-Côte d'Azur,43.296174,5.369953,2025-04-23 10:40:02,19.75,19.09,6.19,50.0,7.20,clear sky
21,Avignon,Avignon,FR,Provence-Alpes-Côte d'Azur,43.949249,4.805901,2025-04-23 10:40:03,19.67,19.02,6.14,51.0,6.17,clear sky
18,Cassis,Cassis,FR,Provence-Alpes-Côte d'Azur,43.214036,5.539632,2025-04-23 10:40:02,19.58,18.90,6.29,50.0,9.08,clear sky


In [34]:
# Enregistrer le DataFrame en fichier CSV
best_weather_df.to_csv('data/top_best_weather.csv', index=False)

In [35]:
# visualiser le top5 des destinations selectionnées
map = folium.Map(location=[46.603354, 1.888334], zoom_start=6)

# Définir le marker
icon_url = "https://images.icon-icons.com/3565/PNG/512/tree_flower_plant_nature_landscape_icon_225325.png"

# Ajouter les villes à la carte avec des icônes personnalisées
for i, row in best_weather_df.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=row['cities'],
        icon=folium.CustomIcon(icon_url, icon_size=(40, 40))  # Ajustez la taille de l'icône si nécessaire
    ).add_to(map)

# Sauvegarder la carte dans un fichier HTML
map.save('map_top_best_destinations.html')

# Afficher la carte
map


# Scraping Booking

In [36]:
best_weather_df

,cities,name,country,state,lat,lon,date,temp,feels_like,uvi,humidity,wind_speed,description
34,Gorges du Verdon,NaN,NaN,NaN,43.749656,6.328562,2025-04-23 10:40:04,19.98,19.42,6.38,53.0,3.53,clear sky
23,Nimes,Nimes,FR,Occitania,43.837425,4.360069,2025-04-23 10:40:03,20.22,19.26,6.22,37.0,4.63,clear sky
19,Marseille,Marseille,FR,Provence-Alpes-Côte d'Azur,43.296174,5.369953,2025-04-23 10:40:02,19.75,19.09,6.19,50.0,7.20,clear sky
21,Avignon,Avignon,FR,Provence-Alpes-Côte d'Azur,43.949249,4.805901,2025-04-23 10:40:03,19.67,19.02,6.14,51.0,6.17,clear sky
18,Cassis,Cassis,FR,Provence-Alpes-Côte d'Azur,43.214036,5.539632,2025-04-23 10:40:02,19.58,18.90,6.29,50.0,9.08,clear sky


## Gorges du Verdon

In [37]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

url_gdv = "https://www.booking.com/searchresults.fr.html?ss=Gorges+du+Verdon&ssne=Gorges+du+Verdon&ssne_untouched=Gorges+du+Verdon&label=gen173nr-1BCAEoggI46AdIM1gEaE2IAQGYAQ24ARfIAQ_YAQHoAQGIAgGoAgO4ApGKo8AGwAIB0gIkN2ViMDZiMzEtODE3Yy00ODM4LTllODctZTNjZGIwY2VlMGQy2AIF4AIB&sid=b10d6df758b0e059011bf7be0724e09f&aid=304142&lang=fr&sb=1&src_elem=sb&src=index&dest_id=2746&dest_type=region&checkin=2025-04-23&checkout=2025-04-24&group_adults=2&no_rooms=1&group_children=0"
url = url_gdv
r = requests.get(url, headers=headers)

response = Selector(text=r.text)

In [38]:
r

<Response [200]>

In [40]:
hotel_name=response.css("div.f6431b446c.a15b38c233 ::text").getall()
hotel_name

['Auberge de Baudinard sur Verdon',
 'La Ferme Rose-Hôtel de Charme',
 'Hôtel Le Colombier',
 'Les Canyons du Verdon',
 'Gabrielle hôtel',
 'Camping La Ferme de Castellane',
 'Lou Paradou',
 'Camping Les Framboiseilles',
 'Camping Arbre de Vie',
 'Villa Borghese, The Originals Relais',
 "Hôtel Spa L'Ermitage",
 'Auberge des Salles',
 'Studi-Hôtel Résidence Plein Voile',
 'Mymoustiers',
 'Gorges du Verdon - Lac Sainte-Croix',
 'Domaine de Majastre',
 'Loft avec mezzanine',
 '" LE CHABASSOLE " CHAMBRES D\'HÔTES - GORGES du VERDON',
 'Le Bosquet',
 'sous les arbres',
 'Hôtel le Panoramic - Votre Hôtel au cœur des Gorges du Verdon',
 'Hôtel La Bonne Auberge',
 'Le Chemin Neuf',
 'Le Chardon 2',
 'Ma Petite Auberge']

In [41]:
hotel_url = response.css("a.a78ca197d0 ::attr(href)").getall()
hotel_url

['https://www.booking.com/hotel/fr/auberge-de-baudinard.fr.html?label=gen173nr-1BCAEoggI46AdIM1gEaE2IAQGYAQ24ARfIAQ_YAQHoAQGIAgGoAgO4ApGKo8AGwAIB0gIkN2ViMDZiMzEtODE3Yy00ODM4LTllODctZTNjZGIwY2VlMGQy2AIF4AIB&sid=b10d6df758b0e059011bf7be0724e09f&aid=304142&ucfs=1&arphpl=1&checkin=2025-04-23&checkout=2025-04-24&dest_id=2746&dest_type=region&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=1&sr_order=popularity&srpvid=5dde4bf9963a18d9&srepoch=1745405301&all_sr_blocks=41142901_170013382_2_34_0&highlighted_blocks=41142901_170013382_2_34_0&matching_block_id=41142901_170013382_2_34_0&sr_pri_blocks=41142901_170013382_2_34_0__10560&from=searchresults',
 'https://www.booking.com/hotel/fr/la-ferme-rose.fr.html?label=gen173nr-1BCAEoggI46AdIM1gEaE2IAQGYAQ24ARfIAQ_YAQHoAQGIAgGoAgO4ApGKo8AGwAIB0gIkN2ViMDZiMzEtODE3Yy00ODM4LTllODctZTNjZGIwY2VlMGQy2AIF4AIB&sid=b10d6df758b0e059011bf7be0724e09f&aid=304142&ucfs=1&arphpl=1&checkin=2025-04-23&checkout=2025-04-24&dest_id=2746&

In [45]:
hotel_ratting= response.css('div.a3b8729ab1.d86cee9b25 ::text').getall()
#hotel_ratting= response.css('div.ac4a7896c7 ::text').getall()
hotel_ratting

['Avec une note de 8,5',
 '8,5',
 'Avec une note de 9,3',
 '9,3',
 'Avec une note de 9,2',
 '9,2',
 'Avec une note de 7,8',
 '7,8',
 'Avec une note de 7,7',
 '7,7',
 'Avec une note de 9,0',
 '9,0',
 'Avec une note de 8,8',
 '8,8',
 'Avec une note de 8,9',
 '8,9',
 'Avec une note de 8,5',
 '8,5',
 'Avec une note de 7,4',
 '7,4',
 'Avec une note de 7,9',
 '7,9',
 'Avec une note de 8,5',
 '8,5',
 'Avec une note de 7,8',
 '7,8',
 'Avec une note de 8,0',
 '8,0',
 'Avec une note de 7,9',
 '7,9',
 'Avec une note de 7,9',
 '7,9',
 'Avec une note de 8,4',
 '8,4',
 'Avec une note de 9,2',
 '9,2',
 'Avec une note de 8,7',
 '8,7',
 'Avec une note de 7,6',
 '7,6',
 'Avec une note de 8,5',
 '8,5',
 'Avec une note de 8,2',
 '8,2',
 'Avec une note de 7,7',
 '7,7',
 'Avec une note de 9,2',
 '9,2',
 'Avec une note de 8,2',
 '8,2']

In [48]:
hotel_ratting = [rating for rating in response.css('div.a3b8729ab1.d86cee9b25 ::text').getall() if "Avec une note de " not in rating]
hotel_ratting

['8,5',
 '9,3',
 '9,2',
 '7,8',
 '7,7',
 '9,0',
 '8,8',
 '8,9',
 '8,5',
 '7,4',
 '7,9',
 '8,5',
 '7,8',
 '8,0',
 '7,9',
 '7,9',
 '8,4',
 '9,2',
 '8,7',
 '7,6',
 '8,5',
 '8,2',
 '7,7',
 '9,2',
 '8,2']

In [39]:

hotel_name=[]
for i in range(100):
    if response.css(f'div.c82435a4b8:nth-child({i}) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > h3:nth-child(1) > a:nth-child(1) > div:nth-child(1)::text').get() is not None:
        hotel_name.append(response.css(f'div.c82435a4b8:nth-child({i}) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > h3:nth-child(1) > a:nth-child(1) > div:nth-child(1)::text').get())
print(hotel_name)


['Auberge de Baudinard sur Verdon', 'La Ferme Rose-Hôtel de Charme', 'Hôtel Le Colombier', 'Les Canyons du Verdon', 'Gabrielle hôtel', 'Camping La Ferme de Castellane', 'Lou Paradou', 'Camping Les Framboiseilles', 'Camping Arbre de Vie', 'Villa Borghese, The Originals Relais', "Hôtel Spa L'Ermitage", 'Auberge des Salles', 'Studi-Hôtel Résidence Plein Voile', 'Mymoustiers', 'Gorges du Verdon - Lac Sainte-Croix', 'Domaine de Majastre', 'Loft avec mezzanine', '" LE CHABASSOLE " CHAMBRES D\'HÔTES - GORGES du VERDON', 'Le Bosquet', 'sous les arbres', 'Hôtel le Panoramic - Votre Hôtel au cœur des Gorges du Verdon', 'Hôtel La Bonne Auberge', 'Le Chemin Neuf', 'Le Chardon 2', 'Ma Petite Auberge']


In [42]:
len(hotel_name)

25

In [43]:
ratting=[]
for i in range(100):
    if response.css(f"div.c82435a4b8:nth-child({i}) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > a:nth-child(1) > span:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1)::text").get() is not None:
        ratting.append(response.css(f"div.c82435a4b8:nth-child({i}) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > a:nth-child(1) > span:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1)::text").get().split()[-1])
print(ratting)


['7,8', '8,3', '7,9', '8,1', '8,2', '8,0', '8,2', '9,2', '8,6', '9,0', '9,6', '7,9', '8,6', '8,4', '8,7', '8,0', '8,9', '9,7', '9,7', '9,2', '7,7', '10']


In [ ]:
import pandas as pd

# Initialisation du DataFrame avec les types de données appropriés
data = {
    'index': pd.Series(dtype='int'),
    'hotel_name': pd.Series(dtype='str'),
    'ratting': pd.Series(dtype='float'),
    'hotel_url' : pd.Series(dtype='str'),
}
df = pd.DataFrame(data)

# Boucle pour remplir le DataFrame
for i in range(100):
    # Ajout de l'index
    df.loc[i, 'index'] = i

    # Récupération et ajout du ratting
    ratting_text = response.css(f"div.c82435a4b8:nth-child({i}) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > a:nth-child(1) > span:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1)::text").get()
    if ratting_text is not None:
        ratting_value = ratting_text.split()[-1].replace(',', '.')  # Remplacer la virgule par un point pour la conversion en float
        df.loc[i, 'ratting'] = float(ratting_value)

    # Récupération et ajout du nom de l'hôtel
    hotel_name_text = response.css(f'div.c82435a4b8:nth-child({i}) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > h3:nth-child(1) > a:nth-child(1) > div:nth-child(1)::text').get()
    if hotel_name_text is not None:
        df.loc[i, 'hotel_name'] = hotel_name_text

# Affichage du DataFrame
df


,index,hotel_name,ratting
0,0.0,NaN,NaN
1,1.0,NaN,NaN
2,2.0,NaN,NaN
3,3.0,NaN,NaN
4,4.0,NaN,NaN
...,...,...,...
95,95.0,NaN,NaN
96,96.0,NaN,NaN
97,97.0,NaN,NaN
98,98.0,NaN,NaN


In [62]:
df=df[~df["hotel_name"].isna()]
df

,index,hotel_name,ratting
5,5.0,Campanile Bayeux,7.8
8,8.0,Le Declic,8.3
9,9.0,Château de Bellefontaine - Teritoria,7.9
11,11.0,Hôtel De Brunville & Spa,8.1
14,14.0,Au Loup Historic Apartments,8.2
15,15.0,Novotel Bayeux,8.0
17,17.0,Hôtel Le Saint Patrice,8.2
19,19.0,Domaine de Bayeux,9.2
24,24.0,Hotel Le Lion D'Or et Restaurant La Table Du Lion,8.6
26,26.0,Logis Les Remparts,9.0


In [87]:
for i in range(len(df)):
    if i < len(hotel_url):
        df.loc[i, 'hotel_url'] = hotel_url[i]
    else:
        df.loc[i, 'hotel_url'] = ""  
df

C:\Users\Myriam\AppData\Local\Temp\ipykernel_8180\2590807557.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,index,hotel_name,ratting,hotel_url
5,5.0,Campanile Bayeux,7.8,https://www.booking.com/hotel/fr/novotel.fr.ht...
8,8.0,Le Declic,8.3,https://www.booking.com/hotel/fr/le-lion-d-or-...
9,9.0,Château de Bellefontaine - Teritoria,7.9,https://www.booking.com/hotel/fr/chambre-d-ha-...
11,11.0,Hôtel De Brunville & Spa,8.1,https://www.booking.com/hotel/fr/le-bayeux.fr....
14,14.0,Au Loup Historic Apartments,8.2,https://www.booking.com/hotel/fr/le-nid-cosy-b...
15,15.0,Novotel Bayeux,8.0,https://www.booking.com/hotel/fr/hotelduluxemb...
17,17.0,Hôtel Le Saint Patrice,8.2,https://www.booking.com/hotel/fr/villa-lara-ha...
19,19.0,Domaine de Bayeux,9.2,https://www.booking.com/hotel/fr/chambre-tout-...
24,24.0,Hotel Le Lion D'Or et Restaurant La Table Du Lion,8.6,https://www.booking.com/hotel/fr/maison-plages...
26,26.0,Logis Les Remparts,9.0,NaN


In [44]:
len(ratting)

22

In [131]:

response.css("div.c82435a4b8:nth-child(8) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div:nth-child(2) > a:nth-child(1)::text").css("a[href]::attr(href)").get()
response.css("html.b_firefox.b_firefox_136.supports_inline-block.supports_flexbox_unprefixed.supports_fontface.supports_hyphens.hasJS body#b2searchresultsPage.bookings2.b2.searchresults.fr.lang_is_ltr.header_reshuffle.no_bg_img.nobg.user_center.app_user_center.sb_gradient_border.b-sprite-3.ds-traveller-header.bigblue_std_sm.bigblue_std_lg.system-font.iq-x-bar.iq-x-bar-new div#bodyconstraint.bodyconstraint--full-width div#bodyconstraint-inner div div div.af5895d4b2 div.df7e6ba27d div.bcbf33c5c3 div.dcf496a7b9.bb2746aad9 div.d4924c9e74 div.c82435a4b8.a178069f51.a6ae3c2b40.a18aeea94d.d794b7a0f7.f53e278e95.c6710787a4 div.c066246e13.d8aec464ca div.c1edfbabcb div.c624d7469d.a0e60936ad.a3214e5942.b0db0e8ada div.b1037148f8.c730b02848 div.a4b53081e1 div.c624d7469d.a0e60936ad.a3214e5942.b23caa1645 div.da8b337763 a.a83ed08757.c21c56c305.a4c1805887.d691166b09.ab98298258.c082d89982.ff33faec5f::text").css("a[href]::attr(href)").get()
response.css("a.a83ed08757.c21c56c305.a4c1805887.d691166b09.ab98298258.c082d89982.ff33faec5f::text").css("a[href]::attr(href)").get()



In [ ]:
html.b_firefox.b_firefox_136.supports_inline-block.supports_flexbox_unprefixed.supports_fontface.supports_hyphens.hasJS body#b2searchresultsPage.bookings2.b2.searchresults.fr.lang_is_ltr.header_reshuffle.no_bg_img.nobg.user_center.app_user_center.sb_gradient_border.b-sprite-3.ds-traveller-header.bigblue_std_sm.bigblue_std_lg.system-font.iq-x-bar.iq-x-bar-new div#bodyconstraint.bodyconstraint--full-width div#bodyconstraint-inner div div div.af5895d4b2 div.df7e6ba27d div.bcbf33c5c3 div.dcf496a7b9.bb2746aad9 div.d4924c9e74 div.c82435a4b8.a178069f51.a6ae3c2b40.a18aeea94d.d794b7a0f7.f53e278e95.c6710787a4 div.c066246e13.d8aec464ca div.c1edfbabcb div.c624d7469d.a0e60936ad.a3214e5942.b0db0e8ada div.b1037148f8.c730b02848 div.a4b53081e1 div.c624d7469d.a0e60936ad.a3214e5942.b23caa1645 div.da8b337763 a.a83ed08757.c21c56c305.a4c1805887.d691166b09.ab98298258.c082d89982.ff33faec5f

In [58]:
print(name)
print(name2)

None
Hôtel Saint Louis


In [ ]:
div.c82435a4b8:nth-child(5) > div:nth-child(1) > div:nth-child(2)

In [40]:
response.css('div.f6431b446c a15b38c233')

[]

In [37]:
xp_ratting="/html/body/div[4]/div/div/div/div[2]/div[3]/div[2]/div[2]/div[3]/div[4]/div[1]/div[2]/div/div[1]/div[2]/div/div/a/span/div/div[1]/div"
ratting = response.xpath(f"{xp_ratting}/text()").get()
ratting = ratting.split()[-1]
ratting

'8,7'

In [80]:
xp_ratting="/html/body/div[4]/div/div/div/div[2]/div[3]/div[2]/div[2]/div[3]/div[4]/div[1]/div[2]/div/div[1]/div[2]/div/div/a/span/div/div[1]/div"
ratting = response.xpath(f"{xp_ratting}/text()").get()

In [242]:
url

'https://www.booking.com/searchresults.fr.html?ss=aigues&efdco=1&label=qwa121jc-1DCAEoggI46AdIM1gDaE2IAQGYAQ24ARfIAQ_YAQPoAQGIAgGoAgO4AoX3mr8GwAIB0gIkMDk1NzlmY2YtNjcwMy00MzdkLWFmNGItMzE3ODQzYzYxNWI32AIE4AIB&sid=f8c0b82bd17e0df0aab4997651af0857&aid=1588662&lang=fr&sb=1&src_elem=sb&src=index&dest_id=-1406800&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=fr&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=9fc26a828ac01775&ac_meta=GhA5ZmMyNmE4MjhhYzAxNzc1IAAoATICZnI6BmFpZ3Vlc0AASgBQAA%3D%3D&checkin=2025-03-28&checkout=2025-03-29&group_adults=2&no_rooms=1&group_children=0'

In [ ]:
xpath = "/html/body/div[4]/div/div/div/div[2]/div[3]/div[2]/div[2]/div[3]/div[4]/div[1]/div[2]/div/div[2]/div[2]/div/div[2]"


In [247]:
response.css("div [class='da8b337763]").css("a[herf]::attr(herf)").get()




SelectorSyntaxError: Unclosed string at 11 (<string>)

In [ ]:
global_endpoint + response.css("nav li[class='next']").css("a[href]::attr(href)").get()

In [ ]:
<span class="e4adce92df">Voir les disponibilités</span><span class="eedba9e88a d7eef963fa"><span class="fcd9eec8fb bf9a32efa5" aria-hidden="true"><span data-testid="availability-cta-icon" class="fcd9eec8fb bf9a32efa5" aria-hidden="true"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" width="50px" data-rtl-flip="true"><path d="M8.913 19.236a.9.9 0 0 0 .642-.266l6.057-6.057a1.3 1.3 0 0 0 .388-.945c.008-.35-.123-.69-.364-.945L9.58 4.966a.91.91 0 0 0-1.284 0 .896.896 0 0 0 0 1.284l5.694 5.718-5.718 5.718a.896.896 0 0 0 0 1.284.88.88 0 0 0 .642.266"></path></svg></span></span></span>

'8,7'

In [223]:
print(hotel_name)
print(ratting)

Hôtel Le Médiéval
8,7


In [225]:
url2 = "https://www.booking.com/hotel/fr/le-medieval.fr.html?aid=1588662&label=qwa121jc-1BCAEoggI46AdIM1gDaE2IAQGYAQ24ARfIAQ_YAQHoAQGIAgGoAgO4AoX3mr8GwAIB0gIkMDk1NzlmY2YtNjcwMy00MzdkLWFmNGItMzE3ODQzYzYxNWI32AIF4AIB&sid=f8c0b82bd17e0df0aab4997651af0857&all_sr_blocks=25855901_400539506_0_2_0&checkin=2025-03-28&checkout=2025-03-29&dest_id=-1406800&dest_type=city&dist=0&group_adults=2&group_children=0&hapos=1&highlighted_blocks=25855901_400539506_0_2_0&hpos=1&matching_block_id=25855901_400539506_0_2_0&no_rooms=1&req_adults=2&req_children=0&room1=A%2CA&sb_price_type=total&sr_order=popularity&sr_pri_blocks=25855901_400539506_0_2_0__8520&srepoch=1743176548&srpvid=72b26b676ce90250&type=total&ucfs=1&"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

r = requests.get(url2, headers=headers)

response = Selector(text=r.text)

In [227]:
xp_desc = "/html/body/div[4]/div/div[4]/div[1]/div[1]/div[2]/div/div[1]/div[1]/div[1]/div/div/p[1]"
description = response.xpath(f"{xp_desc}/text()").get()

In [228]:
description

"L'Hôtel Le Médiéval est situé sur les rives du canal d'Aigues-Mortes, à 30 km de Montpellier. Il propose des chambres pourvues d'une salle de bains privative, d'un parking gratuit et d'une piscine extérieure.\n\nL'Hôtel Le Médiéval propose chaque jour un petit-déjeuner buffet continental qui peut être dégusté sur la terrasse quand le temps le permet. Une connexion Wi-Fi est disponible gratuitement dans les parties communes.\n\nUn parking privé gratuit est disponible sur place. Une promenade de 2 minutes vous mènera aux remparts médiévaux."

In [229]:
xp_desc = "/html/body/div[4]/div/div/div/div[2]/div[3]/div[2]/div[2]/div[3]/div[4]/div[1]/div[2]/div/div[1]/div[1]/div/div[1]/div/h3/a/div[2]"
description = response.xpath(f"{xp_desc}/text()").get()

## La Rochelle

In [85]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

url_LaRochelle = "https://www.booking.com/searchresults.fr.html?ss=La+Rochelle&ssne=La+Rochelle&ssne_untouched=La+Rochelle&label=qwa121jc-1DCAEoggI46AdIM1gDaE2IAQGYAQ24ARfIAQ_YAQPoAQGIAgGoAgO4AoX3mr8GwAIB0gIkMDk1NzlmY2YtNjcwMy00MzdkLWFmNGItMzE3ODQzYzYxNWI32AIE4AIB&sid=b10d6df758b0e059011bf7be0724e09f&aid=1588662&lang=fr&sb=1&src_elem=sb&src=index&dest_id=-1438604&dest_type=city&checkin=2025-04-16&checkout=2025-04-17&group_adults=2&no_rooms=1&group_children=0"
url = url_LaRochelle
r = requests.get(url, headers=headers)

response = Selector(text=r.text)

ConnectTimeout: HTTPSConnectionPool(host='www.booking.com', port=443): Max retries exceeded with url: /searchresults.fr.html?ss=La+Rochelle&ssne=La+Rochelle&ssne_untouched=La+Rochelle&label=qwa121jc-1DCAEoggI46AdIM1gDaE2IAQGYAQ24ARfIAQ_YAQPoAQGIAgGoAgO4AoX3mr8GwAIB0gIkMDk1NzlmY2YtNjcwMy00MzdkLWFmNGItMzE3ODQzYzYxNWI32AIE4AIB&sid=b10d6df758b0e059011bf7be0724e09f&aid=1588662&lang=fr&sb=1&src_elem=sb&src=index&dest_id=-1438604&dest_type=city&checkin=2025-04-16&checkout=2025-04-17&group_adults=2&no_rooms=1&group_children=0 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001AD425E3CD0>, 'Connection to www.booking.com timed out. (connect timeout=None)'))

In [88]:
r

<Response [200]>